In [1]:
! pip install python-dotenv

! pip install openai

! pip install geopy

! pip install azure-cosmos


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Import the required libraries
import time
import os
import json
import uuid
from dotenv import dotenv_values
from openai import OpenAI, AzureOpenAI


#Cosmos DB imports
from azure.cosmos import CosmosClient

In [98]:
env_name = "variable.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

nosql_uri=config["NOSQL_URI"]
cosmos_key = config['NOSQL_PRIMARY_KEY']
cosmos_database = "Account"
cosmos_container = "Transactions"

openai_endpoint = config['AOAI_ENDPOINT']
openai_key = config['AOAI_KEY']
openai_api_version = config['API_VERSION']
openai_embeddings_deployment = config['AOAI_EMBEDDING_DEPLOYMENT']
openai_embeddings_model = config['AOAI_EMBEDDING_DEPLOYMENT_MODEL']
#openai_embeddings_dimensions = int(config['openai_embeddings_dimensions'])

# Create the Azure Cosmos DB for NoSQL client
cosmos_client = CosmosClient(url=nosql_uri, credential=cosmos_key)

azure_openai_embeddings = AzureOpenAI(
    api_version=openai_api_version,
    api_key= openai_key,
    azure_endpoint= openai_endpoint,
)

In [89]:
#create database
DATABASE_NAME = "fraud-nosql-db2"
db= cosmos_client.create_database_if_not_exists(
    id=DATABASE_NAME
)
properties = db.read()
print(json.dumps(properties))

{"id": "fraud-nosql-db2", "_rid": "+39sAA==", "_self": "dbs/+39sAA==/", "_etag": "\"0000f401-0000-4700-0000-66ed8ffd0000\"", "_colls": "colls/", "_users": "users/", "_ts": 1726844925}


In [90]:
vector_embedding_policy = {
    "vectorEmbeddings": [
        {
            "path":"/locationVector",
            "dataType":"float32",
            "distanceFunction":"cosine",
            "dimensions":1536
        },
    ]
}

indexing_policy = {
    "includedPaths": [
        {
            "path": "/*"
        }
    ],
    "excludedPaths": [
        {
            "path": "/\"_etag\"/?"
        },
        {
            "path": "/locationVector/*"
        }
    ],
    "vectorIndexes": [
        {"path": "/locationVector",
         "type": "diskANN"
        }
    ]
}

In [91]:
from azure.cosmos import PartitionKey, exceptions

CONTAINER_NAME = "fraud-nosql-cont"
try:    
    container = db.create_container_if_not_exists(
                    id=CONTAINER_NAME,
                    partition_key=PartitionKey(path="/TenantId"),
                    indexing_policy=indexing_policy,
                    vector_embedding_policy=vector_embedding_policy)

    properties = container.read()
    print('Container with properties \'{0}\' created'.format(properties))

except exceptions.CosmosResourceExistsError:
    print('A container with id \'{0}\' already exists'.format(id))

Container with properties '{'id': 'fraud-nosql-cont', 'indexingPolicy': {'indexingMode': 'consistent', 'automatic': True, 'includedPaths': [{'path': '/*'}], 'excludedPaths': [{'path': '/"_etag"/?'}, {'path': '/locationVector/*'}], 'vectorIndexes': [{'path': '/locationVector', 'type': 'diskANN'}]}, 'partitionKey': {'paths': ['/TenantId'], 'kind': 'Hash', 'version': 2}, 'conflictResolutionPolicy': {'mode': 'LastWriterWins', 'conflictResolutionPath': '/_ts', 'conflictResolutionProcedure': ''}, 'geospatialConfig': {'type': 'Geography'}, 'vectorEmbeddingPolicy': {'vectorEmbeddings': [{'path': '/locationVector', 'dataType': 'float32', 'dimensions': 1536, 'distanceFunction': 'cosine'}]}, '_rid': '+39sAPLryro=', '_ts': 1726844930, '_self': 'dbs/+39sAA==/colls/+39sAPLryro=/', '_etag': '"0000f601-0000-4700-0000-66ed90020000"', '_docs': 'docs/', '_sprocs': 'sprocs/', '_triggers': 'triggers/', '_udfs': 'udfs/', '_conflicts': 'conflicts/'}' created


In [92]:
def generate_embeddings(lat_lon):
    lat_lon_str = f"{lat_lon[0]},{lat_lon[1]}"
    
    # Call OpenAI to generate embeddings (assuming text input is required)
    response = azure_openai_embeddings.embeddings.create(input=lat_lon_str, model=openai_embeddings_model)
    embeddings = response.model_dump()
    
    time.sleep(0.5)  # To avoid API rate limits
    
    return embeddings['data'][0]['embedding']

In [93]:
from geopy.geocoders import Nominatim

def get_city_coordinates(city_name):
    try:
        # Create a geolocator object using Nominatim service
        geolocator = Nominatim(user_agent="MyAPP")
        
        # Geocode the city name to get location details
        location = geolocator.geocode(city_name)
        
        if location:
            # Extract the latitude and longitude from the location object
            lat = location.latitude
            lon = location.longitude
            return lat, lon
        else:
            print(f"City '{city_name}' not found.")
            return None, None
    except Exception as e:
        print(f"Error occurred: {e}")
        return None, None



In [94]:
# Load text-sample_w_embeddings.json which has embeddings pre-computed
data_file = open(file="data/data_with_tenants.json", mode="r") 

data = json.load(data_file)
data_file.close()

In [96]:
# Take a peek at one data item
print(json.dumps(data[4], indent=2))

{
  "TransactionID": "T5109",
  "Amount": 108.79,
  "Timestamp": "2024-09-15 14:02:38",
  "Location": "Boston",
  "Merchant": "Lyft",
  "Fraud": false,
  "TenantId": "5"
}


In [99]:
# Generate embeddings for each location and store data in cosmos db container
for item in data:
    transaction_id = item["TransactionID"]
    item['id'] = transaction_id
    location = item["Location"]
    location_coord = get_city_coordinates(location)
    location_embeddings = generate_embeddings(location_coord)
    item['locationVector'] = location_embeddings
    item['@search.action'] = 'upload'
   
    print("Creating embeddings for transaction:", transaction_id, end='\r')
    
    # Insert the item into the container
    container.upsert_item(item)    

In [132]:
import numpy as np

def get_average_location_vector(container, tenant_id, num_purchases=10):
    # SQL query to get the last 'num_purchases' transactions ordered by timestamp
    sql_query = """
    SELECT TOP @num_purchases c.locationVector
    FROM c
    WHERE c.TenantId = @tenant_id
    ORDER BY c.Timestamp DESC
    """
    
    # Parameters for the query
    parameters = [
        {"name": "@num_purchases", "value": num_purchases}, 
        {"name": "@tenant_id", "value": tenant_id}
    ]
    
    # Execute the query to get the location vectors
    results = container.query_items(
        query=sql_query,
        parameters=parameters,
        enable_cross_partition_query=True
    )
    
    # Collect the location vectors
    vectors = []
    for result in results:
        vectors.append(result['locationVector'])
    
    # If no vectors are found, return None
    if not vectors:
        return None
    
    # Convert the list of vectors into a numpy array
    vectors_np = np.array(vectors)
    
    # Calculate the element-wise average of the vectors
    avg_vector = np.mean(vectors_np, axis=0)
    
    return avg_vector


In [138]:
def vector_search( current_location_vector, tenant_id, average_location_vector, amount, num_results=5):

    if isinstance(current_location_vector, np.ndarray):
        current_location_vector = current_location_vector.tolist()
    if isinstance(average_location_vector, np.ndarray):
        average_location_vector = average_location_vector.tolist()

    sql_query = """
    SELECT TOP @num_results 
        c.TransactionID,
        c.Amount, 
        c.Timestamp, 
        c.Location, 
        c.Merchant,
        c.TenantId, 
        VectorDistance(@average_location_vector, @current_location_vector) AS ProximityofCurrentToAverage
    FROM c
    WHERE 
        VectorDistance(@average_location_vector, @current_location_vector) > 0.1  
        AND c.Amount > @amount * 0.5 
        AND c.Amount < @amount * 2.0
        AND c.TenantId = @tenant_id
    """

    # Parameters for the SQL query
    parameters = [
        {"name": "@num_results", "value": num_results},   
        {"name": "@average_location_vector", "value": average_location_vector},  
        {"name": "@current_location_vector", "value": current_location_vector},  
        {"name": "@amount", "value": amount},  # Transaction amount range filtering
        {"name": "@tenant_id", "value": tenant_id},  # Transaction amount range filtering
    ]

    results = container.query_items(
        query=sql_query,
        parameters=parameters,
        enable_cross_partition_query=True
    )

    return list(results)


In [142]:
tenant_id = '10'
# Assuming you already computed the average location vector and current location vector
average_location_vector = get_average_location_vector(container, tenant_id)

print(average_location_vector)
current_location_vector = generate_embeddings(get_city_coordinates("New york"))
print(current_location_vector)

# Query parameters
query = "Recent purchase at a grocery store"
amount = 10.0  # The current transaction amount

# Perform vector search
results = vector_search(current_location_vector,tenant_id, average_location_vector, amount, num_results=5)
for result in results:
    print(result)


[-0.01492832 -0.0163448   0.00374046 ... -0.00231813 -0.00834704
 -0.01332994]
[-2.139372554665897e-05, -0.005236445460468531, 0.0025807912461459637, 0.021292513236403465, -0.010614736005663872, -0.022411512210965157, -0.02444462478160858, 0.06833777576684952, -0.04249046370387077, 0.0005501419655047357, -0.034547142684459686, 0.06279005855321884, 0.008621024899184704, -0.025469060987234116, 0.013782607391476631, 0.0007372985710389912, -0.0802527591586113, -0.03005538322031498, -0.003311687149107456, 0.005126121453940868, 0.004322332795709372, 0.0007013448048382998, -0.0439719557762146, 0.02917279303073883, -0.0154453469440341, 0.02799075096845627, 0.008668307214975357, -0.01719476841390133, -0.011174235492944717, -0.008841672912240028, 0.009543017484247684, 0.022395752370357513, 0.031253185123205185, -0.007966961711645126, 0.0065879132598638535, 0.015264100395143032, 0.021796850487589836, 0.007801475934684277, 0.0069031245075166225, 0.023089217022061348, 0.02020503580570221, 0.0094484